Data Preparation Phase

In this stage, I will be using pandas to view, explore and clean the dataset.

In [1]:
#imports
import pandas as pd
import ast

In [2]:
df = pd.read_csv("raw_dataset/listings.csv")

In [3]:
df.head(5)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,27934,https://www.airbnb.com/rooms/27934,20250319150518,2025-03-21,city scrape,Nice room with superb city view,Our cool and comfortable one bedroom apartment...,It is very center of Bangkok and easy access t...,https://a0.muscache.com/pictures/566374/23157b...,120437,...,4.91,4.66,4.75,NaN,f,1,1,0,0,0.41
1,27979,https://www.airbnb.com/rooms/27979,20250319150518,2025-03-20,previous scrape,"Easy going landlord,easy place",NaN,NaN,https://a0.muscache.com/pictures/106247594/1d6...,120541,...,NaN,NaN,NaN,NaN,f,2,1,1,0,NaN
2,28745,https://www.airbnb.com/rooms/28745,20250319150518,2025-03-20,previous scrape,modern-style apartment in Bangkok,A modern-style apartment situated in a cool ne...,Ramkumheang neigbourhood - hip place with food...,https://a0.muscache.com/pictures/160369/92a7ec...,123784,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
3,47516,https://www.airbnb.com/rooms/47516,20250319150518,2025-03-20,city scrape,Beautiful waterfront house,NaN,NaN,https://a0.muscache.com/pictures/271998/b090ad...,214456,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
4,48736,https://www.airbnb.com/rooms/48736,20250319150518,2025-03-20,city scrape,Condo with Chaopraya River View,at a size of 25sqm for your vacation in BKK.<b...,NaN,https://a0.muscache.com/pictures/12465296/e9c0...,222005,...,5.00,5.00,5.00,NaN,f,1,0,1,0,0.01


In [4]:
df.rename(columns={"id":"listing_id"},inplace=True)

In [5]:
df.columns

Index(['listing_id', 'listing_url', 'scrape_id', 'last_scraped', 'source',
       'name', 'description', 'neighborhood_overview', 'picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_n

Create the individual dimension table dataframes so that we can start cleaning them.

We will clean the individual dimension tables then merge them back to the original dataframe to obtain the respective primary keys.


In [6]:
hosting_df = df[['host_id', 'host_name', 'host_since', 'host_location', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_identity_verified', 'host_total_listings_count', 'listing_id']].drop_duplicates()
property_df = df[['neighbourhood_cleansed', 'bedrooms', 'bathrooms', 'latitude', 'longitude', 'property_type', 'room_type', 'listing_id']].drop_duplicates()
review_df = df[['number_of_reviews', 
    'review_scores_rating', 
    'review_scores_accuracy',
    'review_scores_cleanliness', 
    'review_scores_checkin', 
    'review_scores_communication', 
    'review_scores_location', 
    'review_scores_value', 
    'listing_id']].drop_duplicates()
amenity_dim = df[['amenities', 'listing_id']].drop_duplicates()

Cleaning the 'hosting' table

In [7]:
hosting_df

,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,host_total_listings_count,listing_id
0,120437,Nuttee,2010-05-08,"Bangkok, Thailand",NaN,NaN,83%,t,t,2.0,27934
1,120541,Emy,2010-05-08,"Bangkok, Thailand",NaN,NaN,NaN,f,f,4.0,27979
2,123784,Familyroom,2010-05-12,"Bangkok, Thailand",NaN,NaN,NaN,f,f,1.0,28745
3,214456,Anuradha,2010-08-27,"New Delhi, India",NaN,NaN,NaN,f,t,2.0,47516
4,222005,Athitaya,2010-09-03,"Bangkok, Thailand",within an hour,100%,100%,f,t,1.0,48736
...,...,...,...,...,...,...,...,...,...,...,...
27543,679097032,Udomchai,2025-02-16,"Bangkok, Thailand",within a few hours,100%,50%,f,t,5.0,1379672358263667520
27544,653148266,Yangzi,2024-09-20,"Bangkok, Thailand",within an hour,100%,41%,f,t,144.0,1379715999009665895
27545,653148266,Yangzi,2024-09-20,"Bangkok, Thailand",within an hour,100%,41%,f,t,144.0,1379720665926456231
27546,653148266,Yangzi,2024-09-20,"Bangkok, Thailand",within an hour,100%,41%,f,t,144.0,1379724086373712054


In [8]:
hosting_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27548 entries, 0 to 27547
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   host_id                    27548 non-null  int64  
 1   host_name                  27545 non-null  object 
 2   host_since                 27546 non-null  object 
 3   host_location              20924 non-null  object 
 4   host_response_time         23482 non-null  object 
 5   host_response_rate         23482 non-null  object 
 6   host_acceptance_rate       23486 non-null  object 
 7   host_is_superhost          26229 non-null  object 
 8   host_identity_verified     27546 non-null  object 
 9   host_total_listings_count  27546 non-null  float64
 10  listing_id                 27548 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 2.3+ MB


In [9]:
hosting_df['host_response_time'].unique()

array([nan, 'within an hour', 'within a few hours', 'a few days or more',
       'within a day'], dtype=object)

In [10]:
hosting_df['host_response_rate'].unique()

array([nan, '100%', '0%', '50%', '40%', '93%', '90%', '33%', '60%', '75%',
       '89%', '97%', '88%', '22%', '20%', '67%', '94%', '17%', '80%',
       '99%', '10%', '91%', '92%', '70%', '83%', '45%', '25%', '82%',
       '44%', '13%', '30%', '86%', '85%', '96%', '14%', '43%', '98%',
       '81%', '95%', '77%', '11%', '79%', '63%', '29%', '73%', '87%',
       '71%', '38%', '78%', '72%', '56%', '57%', '64%', '18%', '84%',
       '69%', '68%'], dtype=object)

In [11]:
hosting_df['host_is_superhost'].unique()

array(['t', 'f', nan], dtype=object)

In [12]:
hosting_df['host_identity_verified'].unique()

array(['t', 'f', nan], dtype=object)

In [13]:
hosting_df['host_is_superhost'] = hosting_df['host_is_superhost'].map({'t':True, 'f':False}).astype('boolean')
hosting_df['host_identity_verified'] = hosting_df['host_identity_verified'].map({'t':True, 'f':False}).astype('boolean')

In [14]:
#Use int16 instead of int64 because it is quite impossible a person has so many listings. 
#Note that we have to use capitalized I otherwise it wont work.
hosting_df['host_total_listings_count'] = hosting_df['host_total_listings_count'].astype('Int16', errors='ignore')

In [15]:
hosting_df['host_since'] = hosting_df['host_since'].astype('datetime64[ns]', errors='ignore')

In [16]:
hosting_df['host_acceptance_rate'] = hosting_df['host_acceptance_rate'].str.rstrip('%').astype(float)/100
hosting_df['host_response_rate'] = hosting_df['host_response_rate'].str.rstrip('%').astype(float)/100

In [17]:
hosting_df

,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,host_total_listings_count,listing_id
0,120437,Nuttee,2010-05-08,"Bangkok, Thailand",NaN,NaN,0.83,True,True,2,27934
1,120541,Emy,2010-05-08,"Bangkok, Thailand",NaN,NaN,NaN,False,False,4,27979
2,123784,Familyroom,2010-05-12,"Bangkok, Thailand",NaN,NaN,NaN,False,False,1,28745
3,214456,Anuradha,2010-08-27,"New Delhi, India",NaN,NaN,NaN,False,True,2,47516
4,222005,Athitaya,2010-09-03,"Bangkok, Thailand",within an hour,1.0,1.00,False,True,1,48736
...,...,...,...,...,...,...,...,...,...,...,...
27543,679097032,Udomchai,2025-02-16,"Bangkok, Thailand",within a few hours,1.0,0.50,False,True,5,1379672358263667520
27544,653148266,Yangzi,2024-09-20,"Bangkok, Thailand",within an hour,1.0,0.41,False,True,144,1379715999009665895
27545,653148266,Yangzi,2024-09-20,"Bangkok, Thailand",within an hour,1.0,0.41,False,True,144,1379720665926456231
27546,653148266,Yangzi,2024-09-20,"Bangkok, Thailand",within an hour,1.0,0.41,False,True,144,1379724086373712054


Cleaning the 'property' table

In [18]:
property_df['property_id'] = range(1, len(property_df)+1)
property_df

,neighbourhood_cleansed,bedrooms,bathrooms,latitude,longitude,property_type,room_type,listing_id,property_id
0,Ratchathewi,1.0,1.5,13.759830,100.541340,Entire condo,Entire home/apt,27934,1
1,Bang Na,NaN,NaN,13.668180,100.616740,Private room in rental unit,Private room,27979,2
2,Bang Kapi,1.0,NaN,13.752320,100.624020,Private room in rental unit,Private room,28745,3
3,Don Mueang,4.0,NaN,13.927260,100.585290,Entire home,Entire home/apt,47516,4
4,Rat Burana,1.0,1.0,13.685560,100.495350,Private room in rental unit,Private room,48736,5
...,...,...,...,...,...,...,...,...,...
27543,Min Buri,1.0,2.0,13.806530,100.736748,Private room in home,Private room,1379672358263667520,27544
27544,Bang Sue,1.0,1.0,13.818700,100.528080,Entire rental unit,Entire home/apt,1379715999009665895,27545
27545,Bang Sue,1.0,1.0,13.818299,100.527187,Entire rental unit,Entire home/apt,1379720665926456231,27546
27546,Bang Sue,1.0,1.0,13.817793,100.528221,Entire rental unit,Entire home/apt,1379724086373712054,27547


In [19]:
property_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27548 entries, 0 to 27547
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neighbourhood_cleansed  27548 non-null  object 
 1   bedrooms                26267 non-null  float64
 2   bathrooms               22115 non-null  float64
 3   latitude                27548 non-null  float64
 4   longitude               27548 non-null  float64
 5   property_type           27548 non-null  object 
 6   room_type               27548 non-null  object 
 7   listing_id              27548 non-null  int64  
 8   property_id             27548 non-null  int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 1.9+ MB


In [20]:
property_df['neighbourhood_cleansed'].unique()

array(['Ratchathewi', 'Bang Na', 'Bang Kapi', 'Don Mueang', 'Rat Burana',
       'Bang Rak', 'Sathon', 'Phaya Thai', 'Khlong Toei', 'Lat Krabang',
       'Parthum Wan', 'Min Buri', 'Khlong San', 'Sai Mai', 'Bangkok Yai',
       'Yan na wa', 'Huai Khwang', 'Vadhana', 'Bang Sue', 'Phra Khanong',
       'Chatu Chak', 'Phasi Charoen', 'Thawi Watthana', 'Bang Kho laen',
       'Thon buri', 'Suanluang', 'Lak Si', 'Thung khru', 'Dusit',
       'Din Daeng', 'Bueng Kum', 'Bang Phlat', 'Saphan Sung', 'Pra Wet',
       'Phra Nakhon', 'Wang Thong Lang', 'Bang Khen', 'Chom Thong',
       'Samphanthawong', 'Khlong Sam Wa', 'Bangkok Noi',
       'Pom Prap Sattru Phai', 'Nong Khaem', 'Lat Phrao', 'Taling Chan',
       'Bang Khun thain', 'Bang Khae', 'Nong Chok', 'Khan Na Yao',
       'Bang Bon'], dtype=object)

In [21]:
property_df['bedrooms'].unique()

array([ 1., nan,  4.,  3.,  2.,  0.,  6.,  9.,  5., 10., 20.,  7., 31.,
        8., 11., 14., 25., 13., 12., 22., 16., 30., 50., 40., 23., 39.,
       15., 46., 17.])

In [22]:
property_df['bathrooms'].unique()

array([ 1.5,  nan,  1. ,  3.5,  2. ,  3. ,  2.5,  0.5,  6. ,  0. ,  5. ,
        4. ,  5.5,  6.5,  4.5,  8. ,  7. ,  7.5,  8.5,  9.5,  9. , 12. ,
       10. , 15. , 11. , 10.5, 12.5, 14. , 13. , 20. , 11.5, 16. , 30. ])

In [23]:
property_df['bedrooms'] = property_df['bedrooms'].astype('Int16')

In [24]:
property_df

,neighbourhood_cleansed,bedrooms,bathrooms,latitude,longitude,property_type,room_type,listing_id,property_id
0,Ratchathewi,1,1.5,13.759830,100.541340,Entire condo,Entire home/apt,27934,1
1,Bang Na,<NA>,NaN,13.668180,100.616740,Private room in rental unit,Private room,27979,2
2,Bang Kapi,1,NaN,13.752320,100.624020,Private room in rental unit,Private room,28745,3
3,Don Mueang,4,NaN,13.927260,100.585290,Entire home,Entire home/apt,47516,4
4,Rat Burana,1,1.0,13.685560,100.495350,Private room in rental unit,Private room,48736,5
...,...,...,...,...,...,...,...,...,...
27543,Min Buri,1,2.0,13.806530,100.736748,Private room in home,Private room,1379672358263667520,27544
27544,Bang Sue,1,1.0,13.818700,100.528080,Entire rental unit,Entire home/apt,1379715999009665895,27545
27545,Bang Sue,1,1.0,13.818299,100.527187,Entire rental unit,Entire home/apt,1379720665926456231,27546
27546,Bang Sue,1,1.0,13.817793,100.528221,Entire rental unit,Entire home/apt,1379724086373712054,27547


Cleaning the 'review' table

In [25]:
review_df['review_id'] = range(1, len(review_df)+1)
review_df

,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,listing_id,review_id
0,65,4.86,4.95,4.82,4.97,4.91,4.66,4.75,27934,1
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27979,2
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28745,3
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47516,4
4,1,5.00,5.00,5.00,5.00,5.00,5.00,5.00,48736,5
...,...,...,...,...,...,...,...,...,...,...
27543,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379672358263667520,27544
27544,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379715999009665895,27545
27545,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379720665926456231,27546
27546,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379724086373712054,27547


In [26]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27548 entries, 0 to 27547
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   number_of_reviews            27548 non-null  int64  
 1   review_scores_rating         18082 non-null  float64
 2   review_scores_accuracy       18082 non-null  float64
 3   review_scores_cleanliness    18081 non-null  float64
 4   review_scores_checkin        18078 non-null  float64
 5   review_scores_communication  18081 non-null  float64
 6   review_scores_location       18077 non-null  float64
 7   review_scores_value          18076 non-null  float64
 8   listing_id                   27548 non-null  int64  
 9   review_id                    27548 non-null  int64  
dtypes: float64(7), int64(3)
memory usage: 2.1 MB


Cleaning the 'amenities' column to enable analysis.

The 'amenities' column currently stores data as a single stringified list per record, making it highly unstructured and unsuitable for database storage and analysis.

This format prevents us from:
1) Comparing listings based on what kind of amenities offered
2) Performing aggregation or filtering with amenities


Standardize_amenities function is used to clean up and standardize the different amenities across listings.

To prevent myself from entering the endless cleaning rabbit hole, I have decided to standardize a few amenities first.

In [27]:
def standardize_amenities(amenities):
    standardized = []
    for amenity in amenities:
        a = amenity.lower().strip() #lower and remove all leading whitespace
        
        if "shampoo" in a:
            standardized.append("shampoo")
        elif "body soap" in a:
            standardized.append("body soap")
        elif "conditioner" in a:
            standardized.append("conditioner")
        elif "refrigerator" in a:
            standardized.append("refrigerator")
        else:
            standardized.append(a)  
    return list(set(standardized)) 

In [28]:
# Convert stringified list to list object
df_copy = df.copy()
df_copy['amenities'] = df_copy['amenities'].apply(ast.literal_eval) #convert the stringified list to actual list
df_copy['amenities'] = df_copy['amenities'].apply(standardize_amenities) #apply standardize function
amenity_df = df_copy[['listing_id','amenities']].explode('amenities')



In [29]:
unique_amenity = amenity_df['amenities'].unique()

In [30]:
unique_amenity_df = pd.DataFrame(unique_amenity)

In [31]:
unique_amenity_df.reset_index(names="amenity_id", inplace=True)
unique_amenity_df.rename(columns={0: "amenity"}, inplace=True)

In [32]:
amenity_df_new = amenity_df.merge(unique_amenity_df, left_on="amenities", right_on="amenity", how="left")

In [33]:
amenity_df_new

,listing_id,amenities,amenity_id,amenity
0,27934,free parking on premises,0,free parking on premises
1,27934,iron,1,iron
2,27934,stove,2,stove
3,27934,wifi,3,wifi
4,27934,kitchen,4,kitchen
...,...,...,...,...
701528,1379757978013222624,outdoor shower,242,outdoor shower
701529,1379757978013222624,hair dryer,10,hair dryer
701530,1379757978013222624,kitchen,4,kitchen
701531,1379757978013222624,washer,11,washer


Connecting the varous dataframes with the listing dataframe

In [34]:
listing_df = df[['listing_id', 'minimum_nights', 'maximum_nights', 'has_availability', 'price', 'accommodates', 'host_id']]

In [35]:
listing_df = listing_df.merge(property_df[['listing_id','property_id']], on="listing_id", how="inner")
listing_df = listing_df.merge(review_df[['listing_id', 'review_id']], on='listing_id', how='inner')
listing_df

,listing_id,minimum_nights,maximum_nights,has_availability,price,accommodates,host_id,property_id,review_id
0,27934,15,240,t,"$1,665.00",2,120437,1,1
1,27979,1,730,NaN,NaN,2,120541,2,2
2,28745,60,730,NaN,NaN,2,123784,3,3
3,47516,3,730,NaN,"$4,373.00",1,214456,4,4
4,48736,14,365,t,"$1,930.00",2,222005,5,5
...,...,...,...,...,...,...,...,...,...
27543,1379672358263667520,2,365,t,$440.00,2,679097032,27544,27544
27544,1379715999009665895,7,365,t,$760.00,2,653148266,27545,27545
27545,1379720665926456231,7,365,t,$760.00,2,653148266,27546,27546
27546,1379724086373712054,7,365,t,$760.00,2,653148266,27547,27547


In [36]:
listing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27548 entries, 0 to 27547
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   listing_id        27548 non-null  int64 
 1   minimum_nights    27548 non-null  int64 
 2   maximum_nights    27548 non-null  int64 
 3   has_availability  25315 non-null  object
 4   price             22158 non-null  object
 5   accommodates      27548 non-null  int64 
 6   host_id           27548 non-null  int64 
 7   property_id       27548 non-null  int64 
 8   review_id         27548 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 1.9+ MB


In [37]:
listing_df['has_availability'] = listing_df['has_availability'].map({"t": True, "f": False}).astype('boolean') 

In [39]:
listing_df['price'] = listing_df['price'].str.lstrip('$').str.replace(',','').astype('float64')

In [40]:
listing_df

,listing_id,minimum_nights,maximum_nights,has_availability,price,accommodates,host_id,property_id,review_id
0,27934,15,240,True,1665.0,2,120437,1,1
1,27979,1,730,<NA>,NaN,2,120541,2,2
2,28745,60,730,<NA>,NaN,2,123784,3,3
3,47516,3,730,<NA>,4373.0,1,214456,4,4
4,48736,14,365,True,1930.0,2,222005,5,5
...,...,...,...,...,...,...,...,...,...
27543,1379672358263667520,2,365,True,440.0,2,679097032,27544,27544
27544,1379715999009665895,7,365,True,760.0,2,653148266,27545,27545
27545,1379720665926456231,7,365,True,760.0,2,653148266,27546,27546
27546,1379724086373712054,7,365,True,760.0,2,653148266,27547,27547


In [42]:
hosting_df.head(5)


,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,host_total_listings_count,listing_id
0,120437,Nuttee,2010-05-08,"Bangkok, Thailand",NaN,NaN,0.83,True,True,2,27934
1,120541,Emy,2010-05-08,"Bangkok, Thailand",NaN,NaN,NaN,False,False,4,27979
2,123784,Familyroom,2010-05-12,"Bangkok, Thailand",NaN,NaN,NaN,False,False,1,28745
3,214456,Anuradha,2010-08-27,"New Delhi, India",NaN,NaN,NaN,False,True,2,47516
4,222005,Athitaya,2010-09-03,"Bangkok, Thailand",within an hour,1.0,1.00,False,True,1,48736
